In [2]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt

In [3]:
bom = pd.read_csv('./data/bom.movie_gross.csv.gz')
tmdb = pd.read_csv('./data/tmdb.movies.csv.gz')
tn = pd.read_csv('./data/tn.movie_budgets.csv.gz')

In [4]:
#looking at the head of each to see where they can be merge
#Also, looking for any columns of interest from each and deciding what to do
bom.head(1)

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788


In [ ]:
tmdb.head(1)

In [ ]:
tn.head(1)

In [ ]:
#creation of the merged table betweeen bom and tmdb
rating_and_gross = bom.merge(tmdb, how='inner', left_on='title', right_on='title').sort_values('domestic_gross', ascending=False).head(20)

In [ ]:
#creation of merge table of the rating_and_gross with the tn table
budget_rating_gross =   rating_and_gross.merge(tn, how='inner', left_on='title', right_on='movie')

In [ ]:
budget_rating_gross.shape


In [ ]:
#dropping unnecessary columns
clean_columns = budget_rating_gross.drop(['Unnamed: 0', 'genre_ids', 'original_title', 'domestic_gross_x', 'foreign_gross', 'movie', 'id_x', 'id_y'], axis=1)
clean_columns

In [ ]:
#check type of each column
clean_columns.info()

In [ ]:
#cleaning and converting worldwide_gross into int
clean_columns['worldwide_gross'] = clean_columns['worldwide_gross'].str.replace('$', '')
clean_columns['worldwide_gross'] = clean_columns['worldwide_gross'].str.replace(',', '')
clean_columns['worldwide_gross'] = clean_columns['worldwide_gross'].astype(int)

In [ ]:
clean_columns['domestic_gross_y'] = clean_columns['domestic_gross_y'].str.replace('$', '')
clean_columns['domestic_gross_y'] = clean_columns['domestic_gross_y'].str.replace(',', '')
clean_columns['domestic_gross_y'] = clean_columns['domestic_gross_y'].astype(int)

In [ ]:
clean_columns['production_budget'] = clean_columns['production_budget'].str.replace('$', '')
clean_columns['production_budget'] = clean_columns['production_budget'].str.replace(',', '')

In [ ]:
#verify that worldwide_gross, domestic_gross_y, and production_budget are int
clean_columns.info()

In [ ]:
#addition of foreign_gross to dataframe
clean_columns['foreign_gross'] = clean_columns['worldwide_gross'] - clean_columns['domestic_gross_y']

In [ ]:
#addition of gross to production budget ratio
clean_columns['gross_to_budget_ratio'] = clean_columns['worldwide_gross'] / clean_columns['production_budget']

In [ ]:
#sorting values by gross_to_budget 
clean_columns.sort_values(by='gross_to_budget_ratio', ascending=False)

In [ ]:
#dropping exact row duplicates
test_clean_columns = clean_columns.drop_duplicates()

In [ ]:
#dropping duplicates where the title name is the same but other values are wrong
vote_count_max = test_clean_columns.groupby(['title','studio']).vote_count.transform(max)
movies_clean = test_clean_columns.loc[test_clean_columns.vote_count == vote_count_max]

In [ ]:
#verifying the shape of movies_clean
movies_clean.shape

In [ ]:
#sorting dataframe by gross_to_budget_ratio
movies_clean.sort_values(['gross_to_budget_ratio'], ascending=False)

In [ ]:
#verifying that there's no null values
movies_clean.info()

In [ ]:
#Quick glance at the information of the dataset
movies_clean.describe()

In [ ]:
#Preping for graphing

import numpy as np


In [ ]:
#MAke to help determining bounds of the graph
q3, q1 = np.percentile(movies_clean['gross_to_budget_ratio'], [75, 25])
q3, q1
iqr = q3-q1

lower_limit = movies_clean['gross_to_budget_ratio'].mean()-(iqr*1.5)

upper_limit = movies_clean['gross_to_budget_ratio'].mean()+(iqr*1.5)

lower_limit, upper_limit

In [ ]:
#limiting data to the upper limit of the iqr
movies_oultierless = movies_clean[movies_clean['gross_to_budget_ratio']<upper_limit]
movies_oultierless

In [ ]:
#graph of movies_oultierless
fig, ax = plt.subplots(figsize=(10,10),)
ax.ticklabel_format(style='plain')
ax.scatter(movies_oultierless['production_budget'] , y = movies_oultierless['worldwide_gross'], alpha = .4);


In [ ]:
#graph for movies_oultierless
np.corrcoef(movies_oultierless['production_budget'] , y = movies_oultierless['worldwide_gross'])
#almost a 1:1 correlation

In [ ]:
#Finding the iqr from movies_without_wwg_outliers
q3, q1 = np.percentile(movies_clean['worldwide_gross'], [75, 25])
q3, q1
iqr = q3-q1

lower_limit_wwg = movies_clean['worldwide_gross'].mean()-(iqr*1.5)

upper_limit_wwg = movies_clean['worldwide_gross'].mean()+(iqr*1.5)

lower_limit_wwg, upper_limit_wwg

In [ ]:
#limiting data to upper limit
movies_without_wwg_outliers = movies_clean[movies_clean['worldwide_gross']<upper_limit_wwg]

In [ ]:
#finding the iqr of the movies_without_wwg_or_pb_outliers
q3, q1 = np.percentile(movies_without_wwg_outliers['production_budget'], [75, 25])
q3, q1
iqr = q3-q1

lower_limit_pb = movies_without_wwg_outliers['production_budget'].mean()-(iqr*1.5)

upper_limit_pb = movies_without_wwg_outliers['production_budget'].mean()+(iqr*1.5)

lower_limit_pb, upper_limit_pb

In [ ]:
#setting the upper limit 
movies_without_wwg_or_pb_outliers = movies_without_wwg_outliers[movies_without_wwg_outliers['production_budget']<upper_limit_pb]

In [ ]:
#graph for movies_without_wwg_or_pb_outliers
fig, ax = plt.subplots(figsize=(10,10),)
ax.ticklabel_format(style='plain')
ax.scatter(movies_without_wwg_or_pb_outliers['production_budget'] , y = movies_without_wwg_or_pb_outliers['worldwide_gross'], alpha = .4);

In [ ]:
#checking the correlation coefficient of movies_without_wwg_or_pb_outliers
np.corrcoef(movies_without_wwg_or_pb_outliers['production_budget'] , y = movies_without_wwg_or_pb_outliers['worldwide_gross'])
#postive correlation but not as strong as the movie_oultierless

In [ ]:
#looking at who the potential competitors are
movies_clean['studio'].value_counts()

In [ ]:
top_ten_most_common_studios = movies_clean['studio'].value_counts().head(20)
top_ten_most_common_studios.index #verifying that there's 20 names

In [ ]:
#visualizing the top_ten_most_common_studios
#tells us how many movies they've made between 2010-2018
fig, ax = plt.subplots(figsize=(10,10))

ax.bar(x=top_ten_most_common_studios.index.astype(str), height=top_ten_most_common_studios);
#most common competitors
#Where is Disney? BV = Disney. Did they buy one of these companies, Who owns who?
#We have a WB and WB (NL), we have Fox and FoxS

In [ ]:
#other EDAs

In [ ]:
#vote_average vs worldwide_gross
fig, ax = plt.subplots(figsize=(10,10),)
ax.ticklabel_format(style='plain')
ax.scatter(movies_clean['vote_average'] , y = movies_clean['worldwide_gross'])

In [ ]:
#looking at correlation
#not that strong, slight positive but overall no reccommend to talk about?
np.corrcoef(movies_clean['vote_average'] , y = movies_clean['worldwide_gross'])

In [ ]:
#popularity vs worldwide_gross
fig, ax = plt.subplots(figsize=(10,10),)
ax.ticklabel_format(style='plain')
ax.scatter(movies_clean['popularity'] , y = movies_clean['worldwide_gross']);

In [ ]:
#decent positive relationship
np.corrcoef(movies_clean['popularity'] , y = movies_clean['worldwide_gross'])

In [ ]:
#vote_count vs vote_average
fig, ax = plt.subplots(figsize=(10,10),)
ax.ticklabel_format(style='plain')
ax.scatter(movies_clean['vote_count'] , y = movies_clean['vote_average']);

In [ ]:
#positve correlation, don't know where .4 stands
np.corrcoef(movies_clean['vote_count'] , y = movies_clean['vote_average'])

In [ ]:
#vote_count vs worldwide_gross
fig, ax = plt.subplots(figsize=(10,10),)
ax.ticklabel_format(style='plain')
ax.scatter(movies_clean['vote_count'] , y = movies_clean['worldwide_gross']);

In [ ]:
#strong positive correlation 
np.corrcoef(movies_clean['vote_count'] , y = movies_clean['worldwide_gross'])

In [ ]:
#looking at how much each studio makes per movie
#some of these studios didn't show up in the most common studios
#meaning they produce less films but make more per
studio_average_wwg = movies_clean.groupby(['studio']).mean(['worldwide_gross']).sort_values(['worldwide_gross'], ascending=False).head(20)
studio_average_wwg

In [ ]:
#studio vs average worldwide_gross per movie
fig, ax = plt.subplots(figsize=(10,10))

ax.bar(x=studio_average_wwg.index.astype(str), height=studio_average_wwg['worldwide_gross']);


In [ ]:
#studio v popularity
studio_popularity = movies_clean.groupby(['studio']).mean(['popularity']).sort_values(['popularity'], ascending=False).head(20)
studio_popularity

In [ ]:
#graph of studio vs popularity
fig, ax = plt.subplots(figsize=(10,10))

ax.bar(x=studio_popularity.index.astype(str), height=studio_popularity['popularity']);

In [ ]:
#studio v vote_average
studio_vote_average = movies_clean.groupby(['studio']).mean(['vote_average']).sort_values(['vote_average'], ascending=False).head(20)
studio_vote_average

In [ ]:
#studio vs vote_average
fig, ax = plt.subplots(figsize=(10,10))

ax.bar(x=studio_vote_average.index.astype(str), height=studio_vote_average['vote_average']);

In [ ]:
#no clue what this is about
studio_vote_count = movies_clean.groupby(['studio']).mean(['vote_count']).sort_values(['vote_count'], ascending=False).head(20)
studio_vote_count

In [ ]:
#studio and vote_count
fig, ax = plt.subplots(figsize=(10,10))

ax.bar(x=studio_vote_count.index.astype(str), height=studio_vote_count['vote_count']);

In [ ]:
import zipfile as zf
import os

In [ ]:
files = zf.ZipFile('./data/im.db.zip', 'r') #these lines create a im.db file which is too big to push
files.extractall() #these lines create a im.db file which is too big to push

In [ ]:
con = sqlite3.connect("./im.db")

In [ ]:
titles = pd.read_sql("""
Select * 

From movie_basics
            
""", con)
titles

In [ ]:
rating = pd.read_sql("""
Select * 

From movie_ratings
            
""", con)

rating

In [ ]:
sql_df = pd.read_sql('''
    SELECT *
    
    FROM
    movie_ratings
    
    INNER JOIN movie_basics
        USING(movie_id) 
     
     WHERE numvotes >= 1000
     
     ORDER BY 
         averagerating DESC
    
    LIMIT 20
''', con)

#We need to choose and averagerating number for a WHERE clause
#If we feel like we need more ratings data we can combine this with budget_rating_gross

In [ ]:
#examining the df for potential analysis and cleaning
sql_df.head()

In [ ]:
#genre column splitting

In [ ]:
#other EDAs

In [ ]:
#genres vs averagerating
genres_average_rating = sql_df.groupby(['genres']).mean(['averagerating']).sort_values(['averagerating'], ascending=False).head(20)
genres_average_rating

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))

ax.bar(x=genres_average_rating.index.astype(str), height=genres_average_rating['averagerating']);

In [ ]:
#runtime_minutes vs start_year
runtime_year = sql_df.groupby(['start_year']).mean(['runtime_minutes']).sort_values(['start_year'])
runtime_year

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))

ax.bar(x=runtime_year.index.astype(str), height=runtime_year['runtime_minutes']);

In [ ]:
#stong positive correlation
#movies are getting longer!
np.corrcoef(runtime_year.index , y = runtime_year['runtime_minutes'])

In [ ]:
con.close() #have this in case file refuses to close

In [ ]:
os.remove('im.db') #This line deletes the im.db file created earlier.